# Logging: cómo generar logs a partir de tu código

La librería estándar de Python `logging` nos permite implementar un sistema de _logs_ o archivos de registro en cualquier script o aplicación que estemos desarrollando.

## ¿Qué es un log?

¿Qué es un log, y qué es el logging? Aquí traducimos un párrafo de la [documentación](https://docs.python.org/3/library/logging.html) de la librería: 

> El _logging_ es un modo de trackear eventos que ocurren a medida que ocurre un determinado _software_. El desarrollador del _software_ añade llamadas de _logging_/registro a su código para indicar que ciertos eventos han ocurrido. Un evento es indicado mediante un mensaje descriptivo, el cual puede contener data variable (i.e. data que es potencialmente distinta cada vez que ocurre el evento). Los eventos también tienen asignada por el desarrollador una importancia, que también puede ser llamada nivel o severidad.

### Diferencia entre un _log_ y un _print_

Muchas veces cuando empezamos a analizar nuestro código usamos comandos _print_ para hacer _debugging_ (por más que hay herramientas especializadas mucho más poderosas para hacerlo). También a veces usamos comandos _print_ para generar, por ejemplo al llamar nuestro script desde la consola, un registro de lo que va ocurriendo que vaya informando al usuario sobre qué está saliendo bien o está saliendo mal.

El concepto de logging es similar a estos dos casos, con la diferencia de que los mensajes no serán impresos en pantalla sino que se guardarán en un archivo de texto (o más de uno, como veremos más abajo) determinado previamente por nosotros.

## Niveles de "severidad"

Como indica el párrafo traducido más arriba, cada mensaje de logging tiene asignado un "nivel" o "severidad". De menor a mayor grado de severidad, son los siguientes.
1. `DEBUG`: se usa para detalles y debugging
2. `INFO`: información sobre el desarrollo (correcto) del proceso
3. `WARNING`: se indica algo inesperado o potencialmente peligroso pero que no impide la ejecución correcta del _software_
4. `ERROR`: algo falló y el _software_ no está ejecutándose como debería
5. `CRITICAL`: error grave.



## Uso básico

Antes de cualquier otra llamada a la librería, debemos comenzar inicializando los parámetros de la librería con `logging.basicConfig`, donde por ejemplo se especifica el nombre del archivo de log. Luego llamamos a la librería para generar los mensajes de nuestro interés. A continuación un ejemplo de uso básico:



In [1]:
import logging

def saludar_comunidad_humai(saludo : bool = True):
    if saludo:
        print("Hola comunidad de Humai!")
    else:
        raise ValueError

def main(saludo : bool):
    logging.basicConfig(filename='mi_archivo_log.log', encoding='utf-8')
    logging.info("Comenzando proceso de saludado")
    try:
        saludar_comunidad_humai(saludo)
    except ValueError:
        logging.warning("El argumento debía ser True. Como pusiste False, se generó una excepción ValueError pero no te preocupes, yo te lo resuelvo ;)")
        print("Hola comunidad de Humai! (uf, eso estuvo cerca)")
    logging.info("Proceso de saludado finalizó correctamente.")

# main(saludo=True) # no genera mensaje de error
# main(saludo=False) # genera mensaje de error

Si prueban descomentando alguna de las dos últimas líneas, verán que los mensajes de nivel `INFO` nunca se escriben en nuestro archivo de log. Esto se debe a que, por defecto, solo los mensajes de nivel `WARNING` para arriba se guardan en el log. Para modificar esto, agregamos el parámetro `level=logging.INFO` a la línea de configuración de la librería. De esta manera, los mensajes de nivel `INFO` para arriba quedarán guardados en el archivo.

Además, podemos hacer que cada mensaje quede etiquetado con la fecha y horario de su creación. Esto requiere modificar el formato de los mensajes, agregando a la línea de configuración el parámetro `format`. Una forma de hacer esto es con `format='%(asctime)s %(levelname)s:%(message)s'`. Nuestro código modificado queda así:

In [2]:
%%python
import logging

def saludar_comunidad_humai(saludo : bool = True):
    if saludo:
        print("Hola comunidad de Humai!")
    else:
        raise ValueError

def main_2(saludo : bool):
    logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s',
                        filename='mi_archivo_log.log', encoding='utf-8',
                        level=logging.INFO)
    logging.info("Comenzando proceso de saludado")
    try:
        saludar_comunidad_humai(saludo)
    except ValueError:
        logging.warning("El argumento debía ser True. Como pusiste False, se generó una excepción ValueError pero no te preocupes, yo te lo resuelvo ;)")
        print("Hola comunidad de Humai! (uf, eso estuvo cerca)")
    logging.info("Proceso de saludado finalizó correctamente.")

# main_2(saludo=True) # no genera mensaje de error
main_2(saludo=False) # genera mensaje de error

Hola comunidad de Humai! (uf, eso estuvo cerca)


**Nota**: ¿Por qué pusimos el comando "mágico"\* `%%python` arriba de todo en la celda anterior? El comando mágico `%%python` indica que toda la celda debe ser ejecutada independientemente en una nueva sesión de python (es como pasarle la celda al comando `python` en la terminal). ¿Por qué hacemos esto?

Resulta que en cada sesión de Python, solo la primera llamada a la función `logging.basicConfig` importa. Es decir que si dentro de este notebook volvemos a llamarla con nuevos parámetros, es lo mismo que no hacer nada. Por eso, para no tener que pedirle al usuario de esta notebook que reinicie el kernel de Jupyter _y evite correr la celda de más arriba_, ejecutamos esta segunda celda en un proceso de Python independiente.

Dicho sea de paso, esta es la misma razón por la cual es fundamental llamar a `logging.basicConfig` antes de generar el primer mensaje de logging, dado que si usamos por ejemplo `logging.error` antes de `logging.basicConfig`, la primera función llama silenciosamente a la segunda con los parámetros por defecto, y por lo tanto la siguiente llamada a `logging.basicConfig` (la nuestra) ya no surtirá efecto.

\* En el contexto de sesiones interactivas de Python, se le dice comandos "mágicos" a aquellos que comienzan con % o %%.


## Ejemplo de uso

En [la primera clase](https://github.com/institutohumai/cursos-python/blob/master/PracticasDeDesarrollo/1_Desarrollo_I/bash_cli_vscode.ipynb) del curso de Desarrollo y Buenas Prácticas en Python de Humai, para aprender a usar `cron` para programar ejecuciones de código, vimos un ejemplo de script que levanta los precios de dos criptomonedas, bitcoin (BTC) y ethereum (ETH). Aquí lo tenemos nuevamente:

In [3]:
%%writefile crypto_price.py
import os
import json
import requests
import pandas as pd
from datetime import datetime

# tenemos que hacer dos funciones: una para recibir el btc y otra para recibir el eth
def btc_price():
    # el endpoint de la API que compara BTC y dólar
    key = "https://api.binance.com/api/v3/ticker/price?symbol=BTCUSDT"
  
    # pedimos la data, la transformamos a dataframe y la devolvemos
    data = requests.get(key)  
    data = data.json()
    btc = pd.DataFrame([data])
    return btc

def eth_price():
    # el endpoint de la API que compara ETH y dólar 
    key = "https://api.binance.com/api/v3/ticker/price?symbol=ETHUSDT"
  
    data = requests.get(key)  
    data = data.json()
    eth = pd.DataFrame([data])
    return eth

def main():
    # armamos una carpeta para almacenar todo, si no existe la crea
    path = 'crypto_prices'
    os.makedirs(path, exist_ok=True)
    
    # al momento de ejecutar esto en consola pedimos los precios de ambas monedas
    btc = btc_price()
    eth = eth_price()
    
    # las unimos
    price = pd.concat([btc, eth])
    
    # identificamos a nivel de segundo cuándo hicimos el request para pegar en el dataframe
    TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
    price['timestamp'] = TIMESTAMP
    
    # armamos el nombre del archivo usando el path con la carpeta que creamos y el timestamp
    FILENAME = f'{path}/prices_{TIMESTAMP}.csv'
    price.to_csv(FILENAME)
    print(f'Data exportada a {FILENAME}')

if __name__ == '__main__':
   main()

Overwriting crypto_price.py


Luego programábamos la ejecución periódica de este script modificando nuestro archivo de configuración crontab. Por ejemplo, para que se ejecute el script una vez por minuto durante los días de semana, el código correspondiente es `* * * * 1-5` (lo cual podemos chequear en [crontab.guru](https://crontab.guru/#*_*_*_*_1-5)), luego debemos agregar a nuestro crontab la línea

`* * * * 1-5 path/a/nuestro/binario/de/python path/a/nuestro/script.py`.

(Si hacen esto en sus sistemas locales, noten que el directorio crypto_prices estará en su carpeta _home_).

Usando la librería `logging` podemos dejar registro de cualquier situación inesperada o error que ocurra durante alguna de las ejecuciones, como por ejemplo un error de conexión al intentar levantar los datos. A continuación, mostramos una manera rápida de _envolver_ el código previo en un entorno `try`/`except`, que justamente maneja el caso en que la función `requests.get` eleva una excepción de tipo `ConnectionError`, agregando el evento al log.

In [6]:
%%writefile crypto_price_logging.py
import logging
import requests
from crypto_price import main as get_crypto_price

def main():
    logging.basicConfig(filename='connection_errors.log', encoding='utf-8')
    try:
      get_crypto_price()
    except requests.exceptions.ConnectionError as e:
        logging.error('Error de conexión: %s', e)
        raise

if __name__ == '__main__':
    main()

Overwriting crypto_price_logging.py


Si ahora desactivamos nuestra conexión a internet y ejecutamos el script desde la consola, veremos que el script deja registrado correctamente que se intentó ejecutar pero hubo un error de conexión, copiando al final el output de dicho error: 

In [5]:
# desconectarse de internet antes de descomentar y correr esta línea!
# !python crypto_price_logging.py

Veremos en nuestro archivo connection_errors.log algo como esto:

![image](imgs/error_de_conexion.png)

Si ahora programamos nuestro script agregándolo al archivo `crontab`, siempre que la ejecución falle por un error de conexión, esto quedará informado en nuestro log.

### Sobre variables en los mensajes de log

El mensaje de error de arriba es un ejemplo de mensaje con datos variables, en este caso la variable `e` contiene el error elevado por el módulo `requests`. En estos casos debemos usar necesariamente la sintaxis vieja para formateo de strings, conocida como "%-style". Debido a cuestiones de compatibilidad, las sintaxis más nuevas requieren una configuración más avanzada de la librería que no cubriremos en esta ocasión.

# Conclusión

Vimos cómo usar la librería `logging` para agregar un registro de errores (u otras informaciones útiles) a nuestro código. La librería permite usos mucho más sofisticados; con respecto a ellos recomendamos leer la [documentación oficial](https://docs.python.org/3/library/logging.html).